# Rellenar días sin registro para la variable
Se rellena el valor del día con la mediana de los valores que se tuvieron para el mismo día y mes en todo el rango y sólo si existen mínimo 10 años

## Cargar librerias

In [1]:
import os
import glob
import pandas as pd
import numpy as np
from datetime import datetime
import time

## Asignar parámetros para el script

In [2]:
variable_path_base = r'../data/variables'
variable_files_pattern = f'var-dia-*.zip'
aggregate_func = 'median'
YEARS_MIN_TO_FILL = 10

## Determinar archivos a procesar

In [3]:
variable_path_base = variable_path_base.split('/')
variable_files = glob.glob(os.path.join(*variable_path_base, variable_files_pattern))
print('Archivos a cargar', *variable_files, sep = '\n')

Archivos a cargar
..\data\variables\var-dia-PTPM_CON.zip
..\data\variables\var-dia-PT_10_TT_D.zip
..\data\variables\var-dia-Q_MEDIA_D.zip
..\data\variables\var-dia-Q_MN_D.zip
..\data\variables\var-dia-Q_MX_D.zip
..\data\variables\var-dia-TMN_CON.zip
..\data\variables\var-dia-TMX_CON.zip


## Leer variable

In [4]:
def read_variable(f):
    dtypes = { 'CodigoEstacion':'category','Etiqueta':'category'}
    dateparse = lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M')
    
    df = pd.read_csv(f, dtype = dtypes, parse_dates = ['Fecha'], date_parser = dateparse)
    return df

## Incluir los días faltantes dejándo Vacío el valor

In [5]:
def fill_missing_days(df):
    #https://stackoverflow.com/a/44978400/1828356
    dates = pd.date_range(start = df.Fecha.min(), end = df.Fecha.max())
    stations = df.CodigoEstacion.unique()
    idx_names = ['CodigoEstacion', 'Fecha']
    idx = pd.MultiIndex.from_product((stations, dates), names = idx_names)
    df = df.set_index(idx_names).reindex(idx)
    df.Etiqueta = etiqueta
    return df

## Rellenar el valor
### Mejoras
* Sólo procesar los días que tuvieron algún vacío
* Sólo rellenar los días en el rango 2000 a 2019

In [6]:
def fill_missing_values(df):
    res = []
    for name, group in df.groupby(['CodigoEstacion']):
        group.loc[:,'ValorFilled'] = group.groupby([group.index.get_level_values('Fecha').strftime('%m-%d')]).Valor.transform(lambda x: x.fillna(x.agg(aggregate_func) if x.count() >= YEARS_MIN_TO_FILL else np.NAN))
        res.append(group)

    df = pd.concat(res)
    return df

## Procesar los archvivos

In [7]:
%%time
for f in variable_files:
    start_time = time.time()
    
    print(f'Archivo: {f}')
    print(f'Leyendo h:{time.time()}...')
    
    df = read_variable(f)
    
    etiqueta = df.iat[0,1]
    
    print(f'**** r:{df.shape} h:{time.time()} t:{(time.time() - start_time) / 60} ****')
    #df.head()
    
    print(f'Llenando días faltantes...')
    df = fill_missing_days(df)
    print(f'**** r:{df.shape} h:{time.time()} t:{(time.time() - start_time) / 60} ****')
    
    print(f'Llenando valores faltantes...')
    df = fill_missing_values(df)
    print(f'**** r:{df.shape} h:{time.time()} t:{(time.time() - start_time) / 60} ****')
    
    print('Completos: ', df[(~df.Valor.isna())].shape)
    print('Rellenados: ', df[(df.Valor.isna()) & (~df.ValorFilled.isna())].shape)
    print('Sin poder rellenar', df[(df.Valor.isna()) & (df.ValorFilled.isna())].shape)
    
    print(f'Almacenando...')
    p = os.path.join(*variable_path_base, f'var-dia-{etiqueta}-fill-{aggregate_func}-{YEARS_MIN_TO_FILL}.csv')
    df.to_csv(p)
    print(f'**** h:{time.time()} t:{(time.time() - start_time) / 60} ****')

Archivo: ..\data\variables\var-dia-PTPM_CON.zip
Leyendo...
**** r:(13764209, 4) t:242.29812145233154 ****
Llenando días faltantes...
**** r:(18694560, 2) t:734.8491775989532 ****
Llenando valores faltantes...


C:\ProgramData\Anaconda3\envs\DS4All_E1\lib\site-packages\pandas\core\indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\ProgramData\Anaconda3\envs\DS4All_E1\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


**** r:(18694560, 3) t:1994.2303085327148 ****
Completos:  (13754048, 3)
Rellenados:  (2124611, 3)
Sin poder rellenar (2815901, 3)
Archivo: ..\data\variables\var-dia-PT_10_TT_D.zip
Leyendo...
**** r:(128414, 4) t:3.099031686782837 ****
Llenando días faltantes...
**** r:(304020, 2) t:7.426452875137329 ****
Llenando valores faltantes...
**** r:(304020, 3) t:29.79794478416443 ****
Completos:  (128414, 3)
Rellenados:  (9313, 3)
Sin poder rellenar (166293, 3)
Archivo: ..\data\variables\var-dia-Q_MEDIA_D.zip
Leyendo...
**** r:(5057682, 4) t:93.57513689994812 ****
Llenando días faltantes...
**** r:(7886096, 2) t:229.07852149009705 ****
Llenando valores faltantes...
**** r:(7886096, 3) t:621.0673246383667 ****
Completos:  (5057682, 3)
Rellenados:  (1521699, 3)
Sin poder rellenar (1306715, 3)
Archivo: ..\data\variables\var-dia-Q_MN_D.zip
Leyendo...
**** r:(2532511, 4) t:69.42765069007874 ****
Llenando días faltantes...
**** r:(5060424, 2) t:144.53154158592224 ****
Llenando valores faltantes...
